# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#load csv from weatherpy
load_file = "output_data/cities.csv"

weather_df = pd.read_csv(load_file)
weather_df

,Unnamed: 0,City,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Country
0,0,Salalah,100,1659489615,100,17.0151,54.0924,75.29,4.61,OM
1,1,Lugovoy,2,1659489736,67,42.9472,72.7644,66.65,9.66,KZ
2,2,Rikitea,40,1659489509,73,-23.1203,-134.9692,72.34,20.58,PF
3,3,Yellowknife,18,1659488981,43,62.4560,-114.3525,62.20,8.32,CA
4,4,Butaritari,20,1659489209,75,3.0707,172.7902,82.02,15.99,KI
...,...,...,...,...,...,...,...,...,...,...
545,545,Ilulissat,75,1659489993,87,69.2167,-51.1000,48.22,1.14,GL
546,546,Genhe,97,1659489993,51,50.7833,121.5167,78.35,3.58,CN
547,547,Nha Trang,100,1659489781,87,12.2500,109.1833,86.00,3.74,VN
548,548,Cap Malheureux,1,1659489642,78,-19.9842,57.6142,66.47,3.00,MU


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#change data types and create variables
humidity = weather_df["Humidity"].astype(float)
maxhumid = humidity.max()
cities = weather_df[["Lat","Lng"]]

gmaps.configure(api_key=g_key)


In [4]:
#create heatmap
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
heat_layer = gmaps.heatmap_layer(cities, weights=humidity, dissipating=False, max_intensity=maxhumid)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#filter data frame for specific conditions
edit_weather_df = weather_df.loc[(weather_df["Max Temp"] > 70) & (weather_df["Max Temp"] < 80) & 
                                 (weather_df["Cloudiness"] == 0) & (weather_df["Wind Speed"] < 10), :]
edit_weather_df = edit_weather_df.dropna(how="any")
edit_weather_df.reset_index(inplace=True)
del edit_weather_df["index"]
del edit_weather_df["Unnamed: 0"]
edit_weather_df

,City,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Country
0,Albany,0,1659489309,64,42.6001,-73.9662,76.64,8.48,US
1,Archidona,0,1659489764,44,37.0965,-4.3887,72.41,4.97,ES
2,Belmonte,0,1659489786,83,-15.8631,-38.8828,70.77,5.57,BR
3,Sochi,0,1659489767,85,43.6000,39.7303,73.49,4.41,RU
4,Mahon,0,1659489801,83,39.8885,4.2658,75.76,2.30,ES
5,Makat,0,1659489811,40,47.6469,53.3463,76.95,8.81,KZ
6,Arrondissement d'Ajaccio,0,1659489758,64,42.0000,8.9167,71.65,8.05,FR
7,Şereflikoçhisar,0,1659489819,56,38.9393,33.5386,73.17,9.04,TR
8,Abu Samrah,0,1659489838,71,35.3029,37.1841,75.24,6.80,SY
9,Tutóia,0,1659489578,88,-2.7619,-42.2744,77.14,9.26,BR


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#create new column for hotels and read in api
hotel_df = edit_weather_df

hotellist = []

for i in range(len(hotel_df)):
    lat = hotel_df.loc[i]['Lat']
    lng = hotel_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
        
hotel_df["Hotel Name"] = hotellist
hotel_df

,City,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Country,Hotel Name
0,Albany,0,1659489309,64,42.6001,-73.9662,76.64,8.48,US,Voorheesville
1,Archidona,0,1659489764,44,37.0965,-4.3887,72.41,4.97,ES,Archidona
2,Belmonte,0,1659489786,83,-15.8631,-38.8828,70.77,5.57,BR,Belmonte
3,Sochi,0,1659489767,85,43.6000,39.7303,73.49,4.41,RU,Bolshoy Sochi
4,Mahon,0,1659489801,83,39.8885,4.2658,75.76,2.30,ES,Mahón
5,Makat,0,1659489811,40,47.6469,53.3463,76.95,8.81,KZ,Makat
6,Arrondissement d'Ajaccio,0,1659489758,64,42.0000,8.9167,71.65,8.05,FR,Sarrola-Carcopino
7,Şereflikoçhisar,0,1659489819,56,38.9393,33.5386,73.17,9.04,TR,Şereflikoçhisar
8,Abu Samrah,0,1659489838,71,35.3029,37.1841,75.24,6.80,SY,Suruj
9,Tutóia,0,1659489578,88,-2.7619,-42.2744,77.14,9.26,BR,Tutóia


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
marker = gmaps.marker_layer(locations)
fig.add_layer(marker)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…